# New York Neighbourhood and Population Data Web scrapping

### Task
Parse the json **nyc_geo.json** into the dataframe with the following columns:
- Borough
- Neighborhood
- Latitude
- Longitude

In [100]:
#import libraries
import pandas as pd
import numpy as np
import json
import datetime
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests

import folium  ## map rendering library

In [80]:
#load json file for neighbourhood
with open('nyc_geo.json') as f:
    nested_json = json.load(f)

df = pd.json_normalize(nested_json)
df

In [81]:
neighbourhood_df = pd.json_normalize(nested_json,record_path ='features')
neighbourhood_df.head()

,type,id,geometry_name,geometry.type,geometry.coordinates,properties.name,properties.stacked,properties.annoline1,properties.annoline2,properties.annoline3,properties.annoangle,properties.borough,properties.bbox
0,Feature,nyu_2451_34572.1,geom,Point,"[-73.84720052054902, 40.89470517661]",Wakefield,1,Wakefield,None,None,0.0,Bronx,"[-73.84720052054902, 40.89470517661, -73.84720..."
1,Feature,nyu_2451_34572.2,geom,Point,"[-73.82993910812398, 40.87429419303012]",Co-op City,2,Co-op,City,None,0.0,Bronx,"[-73.82993910812398, 40.87429419303012, -73.82..."
2,Feature,nyu_2451_34572.3,geom,Point,"[-73.82780644716412, 40.887555677350775]",Eastchester,1,Eastchester,None,None,0.0,Bronx,"[-73.82780644716412, 40.887555677350775, -73.8..."
3,Feature,nyu_2451_34572.4,geom,Point,"[-73.90564259591682, 40.89543742690383]",Fieldston,1,Fieldston,None,None,0.0,Bronx,"[-73.90564259591682, 40.89543742690383, -73.90..."
4,Feature,nyu_2451_34572.5,geom,Point,"[-73.9125854610857, 40.890834493891305]",Riverdale,1,Riverdale,None,None,0.0,Bronx,"[-73.9125854610857, 40.890834493891305, -73.91..."


In [82]:
#split coordinates into longitude and latitude
lat_long_df = pd.DataFrame(neighbourhood_df['geometry.coordinates'].tolist(), columns= ['longitude', 'latitude'])

#check resulting dataframe
lat_long_df.head()

,longitude,latitude
0,-73.847201,40.894705
1,-73.829939,40.874294
2,-73.827806,40.887556
3,-73.905643,40.895437
4,-73.912585,40.890834


In [126]:
#concat lat_long_df, cities_name, borough and borough_boundary

neighbourhood_final = pd.concat([neighbourhood_df['properties.name'], neighbourhood_df['properties.borough'], lat_long_df], axis = 1)

#check new dataframe
neighbourhood_final.head()

,properties.name,properties.borough,longitude,latitude
0,Wakefield,Bronx,-73.847201,40.894705
1,Co-op City,Bronx,-73.829939,40.874294
2,Eastchester,Bronx,-73.827806,40.887556
3,Fieldston,Bronx,-73.905643,40.895437
4,Riverdale,Bronx,-73.912585,40.890834


In [127]:
#rename columns in dataframe
neighbourhood_final.rename(columns={'properties.name': 'Neighbourhood', 'properties.borough':'borough'}, inplace=True)

#check dataframe
neighbourhood_final.head()

,Neighbourhood,borough,longitude,latitude
0,Wakefield,Bronx,-73.847201,40.894705
1,Co-op City,Bronx,-73.829939,40.874294
2,Eastchester,Bronx,-73.827806,40.887556
3,Fieldston,Bronx,-73.905643,40.895437
4,Riverdale,Bronx,-73.912585,40.890834


In [94]:
#check number of boroughs
set(neighbourhood_final['borough'])

{'Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island'}

In [210]:
#check for duplicate neighbourhoods
num_unique_neighborhood = len(set(neighbourhood_final['Neighbourhood']))  #number of unique neighbourhoods
num_neighborhood = len(neighbourhood_final['Neighbourhood'])

duplicate_neighborhood = num_neighborhood - num_unique_neighborhood
duplicate_neighborhood

4

In [129]:
##check duplicated cities
neighbourhood_final[neighbourhood_final.duplicated(['Neighbourhood'], keep=False)]

,Neighbourhood,borough,longitude,latitude
115,Murray Hill,Manhattan,-73.978332,40.748303
116,Chelsea,Manhattan,-74.003116,40.744035
140,Sunnyside,Queens,-73.926916,40.740176
175,Bay Terrace,Queens,-73.776802,40.782843
180,Murray Hill,Queens,-73.812763,40.764126
220,Sunnyside,Staten Island,-74.097126,40.612760
235,Bay Terrace,Staten Island,-74.139166,40.553988
244,Chelsea,Staten Island,-74.189560,40.594726


### Based on research, these are different neighbourhoods that have the same name but their geographical location are different so they belong to different borough. Thus, we will keep all rows of the dataframe

In [98]:
neighbourhood_final.shape

(306, 4)

In [90]:
##convert dataframe to csv
neighbourhood_final.to_csv('neighbourhood_df.csv', index =False)

## Fetch population data of each neighborhood from Wikipedia using BeautifulSoup.

In [101]:
wiki_link = "https://en.wikipedia.org/wiki/Neighborhoods_in_New_York_City"
page = requests.get(wiki_link)
soup = BeautifulSoup(page.text, 'html.parser')

In [156]:
##define a function to get the list of neighbourhoods on wikipedia page
def get_wiki_neighbourhood_df():
    
    '''This function uses BeautifulSoup to scrape wikipedia for the list of neighbourhoods in NYC and returns it as a 
    dataframe containing neighborhood name, anchor tag in wikipedia and population column filled with zeros'''
    
    neighbourhood_df = pd.DataFrame(columns = ['Neighbourhood','Anchor'])
    
    neighbour_list = []
    neighbour_anchor = []
    for i in range(1,60):
        for element in soup.select("table.wikitable tr")[i].findAll('a')[1:]:
            neighbour_list.append(element.text)
            neighbour_anchor.append(element['href'])
            
    neighbourhood_df['Neighbourhood'] = neighbour_list
    neighbourhood_df['Anchor'] = neighbour_anchor
    neighbourhood_df['Population'] = np.zeros(len(neighbour_list))
    neighbourhood_df['ZIP Codes'] = np.zeros(len(neighbour_list))
    
    return neighbourhood_df

In [157]:
wiki_neighbourhood_df = get_wiki_neighbourhood_df()

#check dataframe returned
wiki_neighbourhood_df.head(10)

,Neighbourhood,Anchor,Population,ZIP Codes
0,Melrose,"/wiki/Melrose,_Bronx",0.0,0.0
1,Mott Haven,"/wiki/Mott_Haven,_Bronx",0.0,0.0
2,Port Morris,"/wiki/Port_Morris,_Bronx",0.0,0.0
3,Hunts Point,"/wiki/Hunts_Point,_Bronx",0.0,0.0
4,Longwood,"/wiki/Longwood,_Bronx",0.0,0.0
5,Claremont,"/wiki/Claremont,_Bronx",0.0,0.0
6,Concourse Village,"/wiki/Concourse_Village,_Bronx",0.0,0.0
7,Crotona Park,"/wiki/Crotona_Park,_Bronx",0.0,0.0
8,Morrisania,"/wiki/Morrisania,_Bronx",0.0,0.0
9,Concourse,"/wiki/Concourse,_Bronx",0.0,0.0


In [133]:
wiki_neighbourhood_df.shape

(334, 3)

In [114]:
#Try to extract the population of Chelsea neighbourhood from it's wiki page

neighbourhood_page = requests.get('https://en.wikipedia.org//wiki/Chelsea,_Manhattan')
soup2 = BeautifulSoup(neighbourhood_page.text, 'html.parser')
table = soup2.select("table.infobox tr")

for i in range(len(table)):
    try:
        if 'Population' in table[i].find('th').text:
            print ("Chelsea's population:")
            print (table[i+1].find('td').text)
        else:
            None
    except:
        continue   

Chelsea's population:
47,325


In [155]:
#Try to extract the zip code & population of Chelsea neighbourhood from it's wiki page

neighbourhood_page = requests.get('https://en.wikipedia.org//wiki/Wakefield,_Bronx')
soup2 = BeautifulSoup(neighbourhood_page.text, 'html.parser')
table = soup2.select("table.infobox tr")

for i in range(len(table)):
    try:
        if 'Population' in table[i].find('th').text:
            print ("Wakefield's population:")
            print (table[i+1].find('td').text)
        if 'ZIP Codes' in table[i].find('th').text:
            print ("Wakefield's zip code:")
            print (table[i].find('td').text)
        else:
            None
    except:
        continue   

Wakefield's population:
29,158
Wakefield's zip code:
10466, 10470


In [177]:
##define a function to extract population for all neighbourhoods
def get_neighbourhood_pop(wiki_neighbourhood_df):
    """
    This function replaces the population and ZIP codes of each neighbourhood with values scrapped from Wikipedia
    Input: A dataframe with 4 columns: neighbourhood name, anchor tag in wikipedia, population and ZIP codes
    Output: A dataframe with 4 columns: neighbourhood name, anchor tag in wikipedia, population and ZIP codes
    """
    wiki_search = "https://en.wikipedia.org"

    for i, anchor in enumerate(wiki_neighbourhood_df['Anchor']):
        neighbourhood_page = requests.get(wiki_search+anchor)
        soup2 = BeautifulSoup(neighbourhood_page.text, 'html.parser')
        table = soup2.select("table.infobox tr")
    
        for j in range(len(table)):
            try:
                if 'Population' in table[j].find('th').text: 
                    wiki_neighbourhood_df['Population'][i] = int(table[j+1].find('td').text.replace(',', ''))
                if 'ZIP Codes' in table[j].find('th').text:
                    wiki_neighbourhood_df['ZIP Codes'][i] = table[j].find('td').text
                if 'ZIP Code' in table[j].find('th').text:
                    wiki_neighbourhood_df['ZIP Codes'][i] = table[j].find('td').text
            except:
                pass 
    
    #wiki_neighbourhood_df = wiki_neighbourhood_df[wiki_neighbourhood_df['Population'] != 0]
    
    return wiki_neighbourhood_df

In [178]:
#call the function get_neighbourhood_pop to retrieve population for the neighbourhoods
neighbourhood_pop_df = get_neighbourhood_pop(wiki_neighbourhood_df)

#check dataframe
neighbourhood_pop_df.head(10)

/var/folders/s8/q17ghjw55fn1192037cmrkt00000gn/T/ipykernel_13505/269651025.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wiki_neighbourhood_df['Population'][i] = int(table[j+1].find('td').text.replace(',', ''))
/var/folders/s8/q17ghjw55fn1192037cmrkt00000gn/T/ipykernel_13505/269651025.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wiki_neighbourhood_df['ZIP Codes'][i] = table[j].find('td').text
/var/folders/s8/q17ghjw55fn1192037cmrkt00000gn/T/ipykernel_13505/269651025.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: htt

,Neighbourhood,Anchor,Population,ZIP Codes
0,Melrose,"/wiki/Melrose,_Bronx",24913.0,"10451, 10455, 10456"
1,Mott Haven,"/wiki/Mott_Haven,_Bronx",52413.0,"10451, 10454, 10455"
2,Port Morris,"/wiki/Port_Morris,_Bronx",3523.0,"10454, 10451"
3,Hunts Point,"/wiki/Hunts_Point,_Bronx",12281.0,10474
4,Longwood,"/wiki/Longwood,_Bronx",26196.0,"10455, 10459"
5,Claremont,"/wiki/Claremont,_Bronx",16863.0,"10456, 10459"
6,Concourse Village,"/wiki/Concourse_Village,_Bronx",0.0,"10451, 10452"
7,Crotona Park,"/wiki/Crotona_Park,_Bronx",0.0,0.0
8,Morrisania,"/wiki/Morrisania,_Bronx",16863.0,"10456, 10459"
9,Concourse,"/wiki/Concourse,_Bronx",0.0,"10451, 10452"


In [179]:
neighbourhood_pop_df[neighbourhood_pop_df['Neighbourhood'] == 'Wakefield']

,Neighbourhood,Anchor,Population,ZIP Codes
61,Wakefield,"/wiki/Wakefield,_Bronx",29158.0,"10466, 10470"


In [180]:
#check for null values
neighbourhood_pop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334 entries, 0 to 333
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Neighbourhood  334 non-null    object 
 1   Anchor         334 non-null    object 
 2   Population     334 non-null    float64
 3   ZIP Codes      334 non-null    object 
dtypes: float64(1), object(3)
memory usage: 10.6+ KB


In [268]:
neighbourhood_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Neighbourhood  306 non-null    object 
 1   borough        306 non-null    object 
 2   longitude      306 non-null    float64
 3   latitude       306 non-null    float64
dtypes: float64(2), object(2)
memory usage: 9.7+ KB


In [272]:
#Merge neighbourhoods and population data
NYC_df = neighbourhood_final.merge(neighbourhood_pop_df, how = 'left', on= 'Neighbourhood').drop('Anchor', axis = 1)

#check final dataframe
NYC_df.head()

,Neighbourhood,borough,longitude,latitude,Population,ZIP Codes
0,Wakefield,Bronx,-73.847201,40.894705,29158.0,"10466, 10470"
1,Co-op City,Bronx,-73.829939,40.874294,43752.0,10475
2,Eastchester,Bronx,-73.827806,40.887556,0.0,10466 and 10475
3,Fieldston,Bronx,-73.905643,40.895437,3292.0,10471
4,Riverdale,Bronx,-73.912585,40.890834,48049.0,"10463, 10471"


In [273]:
NYC_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 325 entries, 0 to 324
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Neighbourhood  325 non-null    object 
 1   borough        325 non-null    object 
 2   longitude      325 non-null    float64
 3   latitude       325 non-null    float64
 4   Population     275 non-null    float64
 5   ZIP Codes      275 non-null    object 
dtypes: float64(3), object(3)
memory usage: 17.8+ KB


In [275]:
NYC_df[NYC_df['Population'].isnull()]

,Neighbourhood,borough,longitude,latitude,Population,ZIP Codes
7,Woodlawn,Bronx,-73.867315,40.898273,NaN,NaN
20,High Bridge,Bronx,-73.926102,40.836623,NaN,NaN
30,Country Club,Bronx,-73.824099,40.844246,NaN,NaN
37,North Riverdale,Bronx,-73.904531,40.908543,NaN,NaN
39,Schuylerville,Bronx,-73.826203,40.826580,NaN,NaN
40,Edgewater Park,Bronx,-73.813885,40.821986,NaN,NaN
56,Manhattan Terrace,Brooklyn,-73.957438,40.614433,NaN,NaN
68,Bedford Stuyvesant,Brooklyn,-73.941785,40.687232,NaN,NaN
91,Downtown,Brooklyn,-73.983463,40.690844,NaN,NaN
100,Prospect Park South,Brooklyn,-73.962613,40.647009,NaN,NaN


In [279]:
#separate rows with non-zero and non-null population values
NYC_mod_df = NYC_df[NYC_df['Population'] != 0]
NYC_modified_df = (NYC_mod_df[NYC_mod_df['Population'].notnull()])


#check
NYC_mod_df[NYC_mod_df['Population'] == 0]
NYC_modified_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 165 entries, 0 to 320
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Neighbourhood  165 non-null    object 
 1   borough        165 non-null    object 
 2   longitude      165 non-null    float64
 3   latitude       165 non-null    float64
 4   Population     165 non-null    float64
 5   ZIP Codes      165 non-null    object 
dtypes: float64(3), object(3)
memory usage: 9.0+ KB


In [208]:
#get rows with zero population values
get_pop_df = NYC_df[NYC_df['Population'] == 0]

In [191]:
#convert to csv
get_pop_df.to_csv('get_pop_df.csv', index = False)

In [282]:
#get rows with null population values
missing_pop_df = NYC_mod_df[NYC_mod_df['Population'].isnull()]

missing_pop_df

,Neighbourhood,borough,longitude,latitude,Population,ZIP Codes
7,Woodlawn,Bronx,-73.867315,40.898273,NaN,NaN
20,High Bridge,Bronx,-73.926102,40.836623,NaN,NaN
30,Country Club,Bronx,-73.824099,40.844246,NaN,NaN
37,North Riverdale,Bronx,-73.904531,40.908543,NaN,NaN
39,Schuylerville,Bronx,-73.826203,40.826580,NaN,NaN
40,Edgewater Park,Bronx,-73.813885,40.821986,NaN,NaN
56,Manhattan Terrace,Brooklyn,-73.957438,40.614433,NaN,NaN
68,Bedford Stuyvesant,Brooklyn,-73.941785,40.687232,NaN,NaN
91,Downtown,Brooklyn,-73.983463,40.690844,NaN,NaN
100,Prospect Park South,Brooklyn,-73.962613,40.647009,NaN,NaN


In [287]:
#convert missing_pop_df to csv
missing_pop_df.to_csv('missing_pop.csv', index=False)

In [286]:
print(missing_pop_df.shape)
print(get_pop_df.shape)
print(neighbourhood_final.shape)
print(NYC_modified_df.shape)

(50, 6)
(110, 6)
(306, 4)
(165, 6)


In [288]:
#load neighbourhood population and zip code gotten from niche.com for neighbourhoods zero population values in wikipedia
partial_pop_df = pd.read_csv('manual_pop_zip.csv')

#check dataframe
partial_pop_df.head(10)

,Neighbourhood,borough,longitude,latitude,Population,ZIP Codes
0,Eastchester,Bronx,-73.827806,40.887556,15428.0,"10466, 10475"
1,City Island,Bronx,-73.786488,40.847247,4387.0,10464
2,Westchester Square,Bronx,-73.842194,40.840619,14323.0,"10461, 10462"
3,Van Nest,Bronx,-73.866299,40.843608,15245.0,"10460, 10462"
4,Morris Park,Bronx,-73.850402,40.847549,24243.0,"10461, 10462"
5,Concourse,Bronx,-73.915589,40.834284,75371.0,"10451, 10452"
6,Edenwald,Bronx,-73.848083,40.884561,27491.0,"10466, 10475"
7,Sheepshead Bay,Brooklyn,-73.943186,40.586890,64518.0,"11229, 11235"
8,Kensington,Brooklyn,-73.980421,40.642382,72413.0,11218
9,Kensington,Brooklyn,-73.980421,40.642382,72413.0,11218


In [290]:
#load neighbourhood population and zip code gotten from niche.com for neighbourhoods missing population values in wikipedia
missing_pop_df = pd.read_csv('missing_pop.csv')

#CHECK Dataframe
missing_pop_df.head()

,Neighbourhood,borough,longitude,latitude,Population,ZIP Codes
0,Woodlawn,Bronx,-73.867315,40.898273,9765,10470
1,High Bridge,Bronx,-73.926102,40.836623,37727,10452
2,Country Club,Bronx,-73.824099,40.844246,3548,10465
3,North Riverdale,Bronx,-73.904531,40.908543,48049,10463
4,Schuylerville,Bronx,-73.826203,40.826580,12025,10461


In [291]:
#concat NYC_modified_df and partial_pop_df vertically
NYC_new_df = pd.concat([NYC_modified_df, partial_pop_df], ignore_index = True,sort = False)

#check new dataframe
NYC_new_df      

,Neighbourhood,borough,longitude,latitude,Population,ZIP Codes
0,Wakefield,Bronx,-73.847201,40.894705,29158.0,"10466, 10470"
1,Co-op City,Bronx,-73.829939,40.874294,43752.0,10475
2,Fieldston,Bronx,-73.905643,40.895437,3292.0,10471
3,Riverdale,Bronx,-73.912585,40.890834,48049.0,"10463, 10471"
4,Kingsbridge,Bronx,-73.902818,40.881687,10669.0,10463
...,...,...,...,...,...,...
270,Allerton,Bronx,-73.859319,40.865788,51528.0,10467
271,Kingsbridge Heights,Bronx,-73.901523,40.870392,21408.0,"10463, 10468"
272,Kingsbridge Heights,Bronx,-73.901523,40.870392,21408.0,"10463, 10468"
273,Bayswater,Queens,-73.765968,40.611322,19128.0,11691


In [309]:
#concat NYC_new_df and missing_pop_df vertically
NYC_final_df = pd.concat([NYC_new_df, missing_pop_df], ignore_index = True,sort = False)

#check new dataframe
NYC_final_df.head()

,Neighbourhood,borough,longitude,latitude,Population,ZIP Codes
0,Wakefield,Bronx,-73.847201,40.894705,29158.0,"10466, 10470"
1,Co-op City,Bronx,-73.829939,40.874294,43752.0,10475
2,Fieldston,Bronx,-73.905643,40.895437,3292.0,10471
3,Riverdale,Bronx,-73.912585,40.890834,48049.0,"10463, 10471"
4,Kingsbridge,Bronx,-73.902818,40.881687,10669.0,10463


In [310]:
NYC_final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Neighbourhood  325 non-null    object 
 1   borough        325 non-null    object 
 2   longitude      325 non-null    float64
 3   latitude       325 non-null    float64
 4   Population     325 non-null    float64
 5   ZIP Codes      325 non-null    object 
dtypes: float64(3), object(3)
memory usage: 15.4+ KB


In [311]:
#check for duplicates
#check for duplicate neighbourhoods
num_uniq_neighbourhood = len(set(NYC_final_df['Neighbourhood']))  #number of unique neighbourhoods
num_neighbourhood = len(NYC_final_df['Neighbourhood'])

duplicate_neighbourhood = num_neighbourhood - num_uniq_neighbourhood
duplicate_neighbourhood


23

In [312]:
##check duplicated neighbourhoods
NYC_final_df[NYC_final_df.duplicated(['Neighbourhood'], keep=False)]

,Neighbourhood,borough,longitude,latitude,Population,ZIP Codes
11,University Heights,Bronx,-73.910416,40.855727,25702.0,"10453, 10468"
12,University Heights,Bronx,-73.910416,40.855727,25702.0,"10453, 10468"
35,Bensonhurst,Brooklyn,-73.995180,40.611009,104934.0,"11204, 11214"
36,Bensonhurst,Brooklyn,-73.995180,40.611009,104934.0,"11204, 11214"
39,Gravesend,Brooklyn,-73.973471,40.595260,29436.0,11223[3]
40,Gravesend,Brooklyn,-73.973471,40.595260,29436.0,11223[3]
43,Crown Heights,Brooklyn,-73.943291,40.670829,143000.0,"11213, 11216, 11225, 11233, 11238"
44,Crown Heights,Brooklyn,-73.943291,40.670829,143000.0,"11213, 11216, 11225, 11233, 11238"
66,Midwood,Brooklyn,-73.957595,40.625596,52835.0,"11210, 11230"
67,Midwood,Brooklyn,-73.957595,40.625596,52835.0,"11210, 11230"


In [313]:
# drop rows which have same longitude and latitude and keep the first
NYC_final = NYC_final_df.drop_duplicates(subset = ['Neighbourhood', 'borough'], keep = 'last').reset_index(drop = True)

#check for Neighbourhood duplicates
NYC_final[NYC_final.duplicated(['Neighbourhood'], keep=False)]

,Neighbourhood,borough,longitude,latitude,Population,ZIP Codes
75,Murray Hill,Manhattan,-73.978332,40.748303,10864.0,"10016, 10017"
118,Murray Hill,Queens,-73.812763,40.764126,10864.0,"10016, 10017"
176,Chelsea,Manhattan,-74.003116,40.744035,47325.0,"10001, 10011"
182,Sunnyside,Queens,-73.926916,40.740176,63271.0,11104
190,Bay Terrace,Queens,-73.776802,40.782843,21751.0,"11359, 11360"
210,Sunnyside,Staten Island,-74.097126,40.612760,48840.0,10301
219,Bay Terrace,Staten Island,-74.139166,40.553988,7198.0,"10308, 10306"
226,Chelsea,Staten Island,-74.189560,40.594726,3006.0,10314


### These are different neighbourhoods that have the same name but their geographical location (latitude and longitude) are different so they belong to different borough. Thus, we will keep all rows of the dataframe

In [314]:
NYC_final.shape

(306, 6)

In [315]:
NYC_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Neighbourhood  306 non-null    object 
 1   borough        306 non-null    object 
 2   longitude      306 non-null    float64
 3   latitude       306 non-null    float64
 4   Population     306 non-null    float64
 5   ZIP Codes      306 non-null    object 
dtypes: float64(3), object(3)
memory usage: 14.5+ KB


In [316]:
#check for zero value zip codes
NYC_final[NYC_final['ZIP Codes']== 0]

,Neighbourhood,borough,longitude,latitude,Population,ZIP Codes
58,Marine Park,Brooklyn,-73.931344,40.609748,45231.0,0.0


In [317]:
#replace the zip code of Marine Park
NYC_final.loc[NYC_final['ZIP Codes'] == 0.0, 'ZIP Codes'] = 11234

#check
NYC_final[NYC_final['Neighbourhood']== 'Marine Park']

,Neighbourhood,borough,longitude,latitude,Population,ZIP Codes
58,Marine Park,Brooklyn,-73.931344,40.609748,45231.0,11234


In [318]:
#check for zero value population
NYC_final[NYC_final['Population']== 0]

,Neighbourhood,borough,longitude,latitude,Population,ZIP Codes


In [324]:
#check for zero value zip codes
NYC_final[NYC_final['ZIP Codes']== '0.0']

,Neighbourhood,borough,longitude,latitude,Population,ZIP Codes,MODZCTA
161,Gowanus,Brooklyn,-73.994441,40.673931,20081.0,0.0,0.0


In [326]:
#replace the zip code of Gowanus, Brooklyn
NYC_final.loc[NYC_final['ZIP Codes'] == '0.0', 'ZIP Codes'] = 11215

#check
NYC_final[NYC_final['Neighbourhood']== 'Gowanus']

,Neighbourhood,borough,longitude,latitude,Population,ZIP Codes,MODZCTA
161,Gowanus,Brooklyn,-73.994441,40.673931,20081.0,11215,0.0


In [327]:
#convert dataframe to csv
NYC_final.to_csv('NYC_neighbourhood.csv', index=False)

### Task
Use different data sources and APIs to collect information about the neigborhoods that can be used for segmentation.

In [228]:
#load zip_code to zcta
zip_to_zcta = pd.read_csv('updated22-Table 1.csv')

#check
zip_to_zcta.head()

,ZIP_CODE,PO_NAME,STATE,ZCTA
0,501,Holtsville,NY,11742
1,544,Holtsville,NY,11742
2,10001,New York,NY,10001
3,10002,New York,NY,10002
4,10003,New York,NY,10003


In [230]:
#load zcta to MODZCTA file
zcta_to_modzcta = pd.read_csv('Sheet 1-Table 1.csv')

#check
zcta_to_modzcta.head()

,ZCTA,MODZCTA
0,10001,10001
1,10002,10002
2,10003,10003
3,10004,10004
4,10005,10005


In [240]:
#left merge zcta_to_modzcta with zip_to_zcta
modzcta_mapping = pd.merge(zcta_to_modzcta,zip_to_zcta, how = 'inner', on='ZCTA')

#check
modzcta_mapping 

,ZCTA,MODZCTA,ZIP_CODE,PO_NAME,STATE
0,10001,10001,10001,New York,NY
1,10001,10001,10118,New York,NY
2,10001,10001,10120,New York,NY
3,10001,10001,10122,New York,NY
4,10001,10001,10123,New York,NY
...,...,...,...,...,...
308,11692,11692,11692,Arverne,NY
309,11693,11693,11693,Far Rockaway,NY
310,11693,11693,11695,Far Rockaway,NY
311,11694,11694,11694,Rockaway Park,NY


In [242]:
#check dataframe merging
modzcta_mapping [modzcta_mapping ['ZCTA'] == 10001]

,ZCTA,MODZCTA,ZIP_CODE,PO_NAME,STATE
0,10001,10001,10001,New York,NY
1,10001,10001,10118,New York,NY
2,10001,10001,10120,New York,NY
3,10001,10001,10122,New York,NY
4,10001,10001,10123,New York,NY
5,10001,10001,10125,New York,NY
6,10001,10001,10130,New York,NY


In [390]:
#load NYC neighbourhood dataframe
NYC_pop_zip_df = pd.read_csv('NYC_neighbourhood.csv')

#check dataframe
NYC_pop_zip_df.head()

,Neighbourhood,borough,longitude,latitude,Population,ZIP Codes
0,Wakefield,Bronx,-73.847201,40.894705,29158.0,"10466, 10470"
1,Co-op City,Bronx,-73.829939,40.874294,43752.0,10475
2,Fieldston,Bronx,-73.905643,40.895437,3292.0,10471
3,Riverdale,Bronx,-73.912585,40.890834,48049.0,"10463, 10471"
4,Kingsbridge,Bronx,-73.902818,40.881687,10669.0,10463


In [391]:
#split ZIP Codes column to obtain a single zip code per neighbourhood
NYC_pop_zip_df[['ZIP_CODE','ZIP_1', 'ZIP_2', 'ZIP_3', 'ZIP_4', 'ZIP_5', 'ZIP_6', 'ZIP_7']]= NYC_pop_zip_df['ZIP Codes'].str.split(',', expand=True)

#drop all zip columns except the first
NYC_pop_zip_df.drop(['ZIP_1', 'ZIP_2', 'ZIP_3', 'ZIP_4', 'ZIP_5', 'ZIP_6', 'ZIP_7'], axis = 1, inplace=True)

#check
NYC_pop_zip_df

,Neighbourhood,borough,longitude,latitude,Population,ZIP Codes,ZIP_CODE
0,Wakefield,Bronx,-73.847201,40.894705,29158.0,"10466, 10470",10466
1,Co-op City,Bronx,-73.829939,40.874294,43752.0,10475,10475
2,Fieldston,Bronx,-73.905643,40.895437,3292.0,10471,10471
3,Riverdale,Bronx,-73.912585,40.890834,48049.0,"10463, 10471",10463
4,Kingsbridge,Bronx,-73.902818,40.881687,10669.0,10463,10463
...,...,...,...,...,...,...,...
301,Sandy Ground,Staten Island,-74.217766,40.541140,33883.0,10309,10309
302,Roxbury,Queens,-73.892138,40.567376,2170.0,11697,11697
303,Lighthouse Hill,Staten Island,-74.137927,40.576506,8849.0,10306,10306
304,Hammels,Queens,-73.805530,40.587338,13029.0,11693,11693


In [392]:
NYC_pop_zip_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Neighbourhood  306 non-null    object 
 1   borough        306 non-null    object 
 2   longitude      306 non-null    float64
 3   latitude       306 non-null    float64
 4   Population     306 non-null    float64
 5   ZIP Codes      306 non-null    object 
 6   ZIP_CODE       306 non-null    object 
dtypes: float64(3), object(4)
memory usage: 16.9+ KB


In [393]:
#convert ZIP_CODE column to int64 inorder to merge
NYC_pop_zip_df['ZIP_CODE'] = NYC_pop_zip_df['ZIP_CODE'].astype(int)

In [394]:
#left merge NYC_pop_zip_df with modzcta_mapping to get MODZCTA for each neighbourhood
NYC_pop_modzcta = NYC_pop_zip_df.merge(modzcta_mapping, how='left', on='ZIP_CODE')

#check
NYC_pop_modzcta

,Neighbourhood,borough,longitude,latitude,Population,ZIP Codes,ZIP_CODE,ZCTA,MODZCTA,PO_NAME,STATE
0,Wakefield,Bronx,-73.847201,40.894705,29158.0,"10466, 10470",10466,10466,10466,Bronx,NY
1,Co-op City,Bronx,-73.829939,40.874294,43752.0,10475,10475,10475,10475,Bronx,NY
2,Fieldston,Bronx,-73.905643,40.895437,3292.0,10471,10471,10471,10471,Bronx,NY
3,Riverdale,Bronx,-73.912585,40.890834,48049.0,"10463, 10471",10463,10463,10463,Bronx,NY
4,Kingsbridge,Bronx,-73.902818,40.881687,10669.0,10463,10463,10463,10463,Bronx,NY
...,...,...,...,...,...,...,...,...,...,...,...
301,Sandy Ground,Staten Island,-74.217766,40.541140,33883.0,10309,10309,10309,10309,Staten Island,NY
302,Roxbury,Queens,-73.892138,40.567376,2170.0,11697,11697,11697,11697,Breezy Point,NY
303,Lighthouse Hill,Staten Island,-74.137927,40.576506,8849.0,10306,10306,10306,10306,Staten Island,NY
304,Hammels,Queens,-73.805530,40.587338,13029.0,11693,11693,11693,11693,Far Rockaway,NY


In [395]:
#drop columns ZIP Codes, ZCTA, PO_NAME, STATE
NYC_pop_modzcta.drop(['ZIP Codes', 'ZCTA', 'PO_NAME', 'STATE'], axis = 1, inplace=True)

In [396]:
#check dataframe
NYC_pop_modzcta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306 entries, 0 to 305
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Neighbourhood  306 non-null    object 
 1   borough        306 non-null    object 
 2   longitude      306 non-null    float64
 3   latitude       306 non-null    float64
 4   Population     306 non-null    float64
 5   ZIP_CODE       306 non-null    int64  
 6   MODZCTA        306 non-null    int64  
dtypes: float64(3), int64(2), object(2)
memory usage: 19.1+ KB


In [397]:
#check new dataframe
NYC_pop_modzcta.head(10)

,Neighbourhood,borough,longitude,latitude,Population,ZIP_CODE,MODZCTA
0,Wakefield,Bronx,-73.847201,40.894705,29158.0,10466,10466
1,Co-op City,Bronx,-73.829939,40.874294,43752.0,10475,10475
2,Fieldston,Bronx,-73.905643,40.895437,3292.0,10471,10471
3,Riverdale,Bronx,-73.912585,40.890834,48049.0,10463,10463
4,Kingsbridge,Bronx,-73.902818,40.881687,10669.0,10463,10463
5,Marble Hill,Manhattan,-73.910660,40.876551,9481.0,10463,10463
6,Norwood,Bronx,-73.879391,40.877224,40494.0,10467,10467
7,Williamsbridge,Bronx,-73.857446,40.881039,61321.0,10466,10466
8,Baychester,Bronx,-73.835798,40.866858,63345.0,10469,10469
9,Pelham Parkway,Bronx,-73.854756,40.857413,30073.0,10461,10461
